In [2]:
import time
import warnings
from time import time

import dask.dataframe as dd
import dask_geopandas as dask_geopandas
import geopandas
import libpysal
import momepy
import numpy as np
import pandas as pd
import pygeos
import scipy
import networkx as nx
from dask.distributed import Client, LocalCluster, as_completed
from libpysal.weights import Queen
from momepy_utils import (
    _circle_radius,
    centroid_corner,
    elongation,
    get_corners,
    get_edge_ratios,
    get_nodes,
    solar_orientation_poly,
    squareness,
)

In [3]:
def fix(chunk_id):
    s = time()
    cells = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq")
    cells['keep'] = True

    # add neighbouring cells from other chunks
    cross_chunk_cells = []

    for chunk, inds in cross_chunk.loc[chunk_id].indices.iteritems():
        add_cells = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk}.pq").iloc[inds]
        add_cells['keep'] = False
        cross_chunk_cells.append(add_cells)

    df = cells.append(pd.concat(cross_chunk_cells, ignore_index=True), ignore_index=True)

    # read W
    w = libpysal.weights.WSP(scipy.sparse.load_npz(f"../../urbangrammar_samba/spatial_signatures/weights/w_{chunk_id}.npz")).to_W()

    # read W3
    w3 = libpysal.weights.WSP(scipy.sparse.load_npz(f"../../urbangrammar_samba/spatial_signatures/weights/w3_{chunk_id}.npz")).to_W()

    # mean interbuilding distance
    # define adjacency list from lipysal
    adj_list = w.to_adjlist(remove_symmetric=False)
    adj_list["weight"] = (
        df.buildings.iloc[adj_list.focal]
        .reset_index(drop=True)
        .distance(df.buildings.iloc[adj_list.neighbor].reset_index(drop=True)).values
    )

    G = libpysal.weights.W.from_adjlist(adj_list).to_networkx()
    ibd = []
    for i in range(len(df)):
        try:
            sub = nx.ego_graph(G, i, radius=3)
            ibd.append(np.nanmean([x[-1] for x in list(sub.edges.data('weight'))]))
        except:
            ibd.append(np.nan)

    df['ltbIBD'] = ibd
    
    df['stbCeA'] = (df['stbOri'] - df['stcOri']).abs()

    df[df['keep']].drop(columns=['keep']).to_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq")
    
    return f"Chunk {chunk_id} processed sucessfully in {time() - s} seconds."


In [4]:
workers = 8
client = Client(LocalCluster(n_workers=workers, threads_per_worker=1))
client

Client Scheduler: tcp://127.0.0.1:44035 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 84.28 GB


In [5]:
cross_chunk = pd.read_parquet('../../urbangrammar_samba/spatial_signatures/cross-chunk_indices.pq')

In [6]:
inputs = iter(range(103))
futures = [client.submit(fix, next(inputs)) for i in range(workers)]
ac = as_completed(futures)
for finished_future in ac:
    # submit new future 
    try:
        new_future = client.submit(fix, next(inputs))
        ac.add(new_future)
    except StopIteration:
        pass
    print(finished_future.result())

Chunk 0 processed sucessfully in 447.46562814712524 seconds.
Chunk 3 processed sucessfully in 507.42181158065796 seconds.
Chunk 2 processed sucessfully in 573.2159600257874 seconds.
Chunk 1 processed sucessfully in 613.3537275791168 seconds.
Chunk 7 processed sucessfully in 672.8189880847931 seconds.
Chunk 4 processed sucessfully in 755.1138219833374 seconds.
Chunk 5 processed sucessfully in 953.9436428546906 seconds.
Chunk 9 processed sucessfully in 571.8970487117767 seconds.
Chunk 11 processed sucessfully in 499.3399028778076 seconds.
Chunk 10 processed sucessfully in 550.8861644268036 seconds.
Chunk 8 processed sucessfully in 715.9609048366547 seconds.
Chunk 6 processed sucessfully in 1273.713731765747 seconds.
Chunk 13 processed sucessfully in 656.2928922176361 seconds.
Chunk 12 processed sucessfully in 871.6294603347778 seconds.
Chunk 15 processed sucessfully in 533.1612844467163 seconds.
Chunk 14 processed sucessfully in 663.3884124755859 seconds.
Chunk 17 processed sucessfully i

In [7]:
client.close()